In [4]:
from sklearn.linear_model import LinearRegression
import csv
import matplotlib.pyplot as plt
import pandas as pd

data = []

with open('clusters.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    headers = next(reader)
    for row in reader:
        data.append([float(i) for i in row])
df = pd.DataFrame(data[1:], columns = headers)
df

,temp.niche.low,temp.niche.high,temp.max.rate,temp.temp.at.max.rate,water.niche.low,water.high,water.max.r,water.mpa.at.max.r,ext_rate_10,ext_rate_16,ext_rate_22,decomp_rate_mean,ranking
0,15.140000,32.680000,11.885342,28.540000,0.200000,1.764000,9.888537,-0.428000,3.224000,6.880000,8.762000,17.648000,0.878003
1,19.057143,32.157143,7.077370,29.228571,0.327143,1.944286,4.284339,-0.597143,0.948571,2.375714,5.042857,6.007143,0.515761
2,16.577778,31.255556,1.313477,26.944444,0.247778,2.420000,1.116078,-0.536667,0.321111,0.583333,1.078889,3.467778,0.277910
3,16.600000,34.200000,12.161347,30.400000,0.120000,1.310000,11.397738,-0.260000,3.300000,5.850000,8.670000,34.010000,0.788430
4,20.800000,30.100000,7.549717,27.300000,0.100000,1.290000,4.777815,-0.240000,0.360000,1.280000,4.620000,21.870000,0.284747
5,20.400000,37.625000,10.646496,33.425000,0.222500,1.685000,4.696383,-0.425000,1.442500,2.792500,5.700000,13.105000,0.634340


In [5]:
results = [['cluster', 'm_l', 'b_l', 'm_r', 'b_r', 'max_rate_temp']]

for ind, cluster in enumerate(data):
    max_rate_temp = cluster[3]
    max_rate = cluster[2]
    #low, high, at max rate, 10, 16, 22
    d = {'temp': [cluster[0], cluster[1], max_rate_temp, 10, 16, 22], 'rate': [max_rate/2, max_rate/2, max_rate, cluster[8], cluster[9], cluster[10]]}
    df_c = pd.DataFrame(data=d)
    df_left = df_c[df_c['temp'] <= max_rate_temp]
    df_right = df_c[df_c['temp'] >= max_rate_temp]
    
    lr_left = LinearRegression()
    lr_left.fit([[x] for x in df_left['temp']], df_left['rate'])
    lr_right = LinearRegression()
    lr_right.fit([[x] for x in df_right['temp']], df_right['rate'])
    results.append([ind, lr_left.coef_[0], lr_left.intercept_, lr_right.coef_[0], lr_right.intercept_, max_rate_temp])

In [6]:
with open('rate_temp_fit.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in results:
        writer.writerow(row)

In [7]:
results = [['cluster', 'm_l', 'b_l', 'm_r', 'b_r', 'max_rate_water']]

for ind, cluster in enumerate(data):
    x = [cluster[4], cluster[5], cluster[7]] #low, high, at max rate
    m_l = .5/(x[2]-x[0])
    b_l = 1 - m_l*x[2]
    m_r = .5/(x[1]-x[2])
    b_r = 1 - m_r*x[2]
    results.append([ind, m_l, b_l, m_r, b_r, cluster[7]])

In [8]:
with open('percentmaxr_water_fit.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in results:
        writer.writerow(row)